In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.InteractiveSession()

In [3]:
data = np.load('p1.npy').item()

In [4]:
num_cam = 3
num_features = 72
num_hidden = 2048

x2d = data['joints']
q3d = data['theta'][:, num_cam:(num_cam + num_features)]
num_steps = x2d.shape[0]

In [24]:
lr = 0.1

In [25]:
data.keys()

['cams', 'joints', 'verts', 'joints3d', 'theta']

# Note
Latent features: $Z=\{z_t\}$

Loss function: $l_{rec}(Z)=w_{2D}l_{2D}(Z)+ w_{3D}l_{3D}(Z)+w_{sm}l_{sm}(Z)$

2D loss:
$$
l_{2D}=\sum_t\sum_j c_{t,j}\left\Vert\hat{x}_{t,j}-\Pi[F_j(q(z_t))]\right\Vert_1
$$

3D loss:
$$
l_{3D}= \sum_t w_t dist(\hat{q}_t, q(z_t))
$$

Smooth loss:
$$
l_{sm}= \sum_t \sum_j \left\Vert F_j(q(z_t))-F_j(q(z_{t+1}))\right\Vert_2^2
$$



In [26]:
y_true = data['joints']

In [27]:
init = tf.constant(np.random.rand(1, 2))
z = tf.get_variable("Z", shape=(num_steps, num_hidden), initializer=tf.truncated_normal_initializer())

ValueError: Variable Z already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-7-a8b3bd5a7afd>", line 1, in <module>
    z = tf.get_variable("Z", shape=(num_steps, num_hidden), initializer=tf.truncated_normal_initializer())
  File "/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2878, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2818, in run_ast_nodes
    if self.run_code(code, result):


In [28]:
def F(z):
    weights = tf.random_normal([2048, 19* 2])
    out = tf.matmul(z, weights)
    out = tf.reshape(out, [-1, 19, 2])
    return out

In [29]:
y = F(z)

In [30]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y)
optimizer = tf.train.GradientDescentOptimizer(lr)
train = optimizer.minimize(loss)

In [31]:
sess.run(tf.global_variables_initializer())

In [32]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  if i % 10 == 0:
        print("step {}, loss = {}".format(i, loss_value))

step 0, loss = 12837.4785156
step 10, loss = 12093.0654297
step 20, loss = 12263.2255859
step 30, loss = 12877.0058594
step 40, loss = 12698.1933594
step 50, loss = 12799.0566406
step 60, loss = 12833.2949219
step 70, loss = 12813.4970703
step 80, loss = 12375.6660156
step 90, loss = 12923.4521484


In [33]:
sess.run(y)

array([[[ -3.6404862 ,  32.225452  ],
        [-47.85466   ,  25.889977  ],
        [-30.617075  , -25.52144   ],
        ...,
        [ 30.628723  ,  47.577682  ],
        [ 31.394442  ,   5.0311365 ],
        [ 67.79482   ,  41.15177   ]],

       [[ -9.090286  ,  27.360525  ],
        [ 16.829334  , -24.961092  ],
        [  4.5433803 , -67.196785  ],
        ...,
        [ 14.183479  , -44.080204  ],
        [-25.926945  ,  -6.193497  ],
        [ 38.99374   ,  36.168922  ]],

       [[  3.3253174 ,  58.373287  ],
        [-47.902878  , -49.32214   ],
        [ 14.001379  ,  50.870667  ],
        ...,
        [ 66.051     ,  10.2701    ],
        [ 67.97774   ,  86.33458   ],
        [  4.8336964 ,  32.25796   ]],

       ...,

       [[ 30.944813  , -13.981377  ],
        [-67.62806   , -32.92884   ],
        [-48.59563   ,  11.181901  ],
        ...,
        [ 35.086243  , -51.09581   ],
        [ -0.47501087,  18.620335  ],
        [ 66.721634  , -46.696495  ]],

       [[  2.01

In [34]:
data['joints']

array([[[ 70.74985 , 175.8284  ],
        [ 90.632996, 146.16658 ],
        [102.96744 , 110.11203 ],
        ...,
        [114.08213 ,  51.3454  ],
        [119.958595,  48.869904],
        [106.5837  ,  49.36264 ]],

       [[ 68.30115 , 171.32574 ],
        [ 89.460304, 134.67717 ],
        [101.52212 ,  98.315346],
        ...,
        [139.46268 ,  54.233665],
        [138.35754 ,  46.919487],
        [131.30118 ,  47.870167]],

       [[ 72.433914, 173.05396 ],
        [ 89.881966, 137.26054 ],
        [100.6772  , 100.550766],
        ...,
        [134.31285 ,  53.31913 ],
        [136.12846 ,  49.335995],
        [127.547165,  45.80071 ]],

       ...,

       [[ 74.01258 , 175.23819 ],
        [ 84.519775, 150.6713  ],
        [ 90.922455, 119.5243  ],
        ...,
        [108.92292 ,  69.0672  ],
        [114.25256 ,  73.73198 ],
        [103.403564,  70.796844]],

       [[ 88.34883 , 164.2773  ],
        [ 99.79211 , 139.94206 ],
        [103.388214, 107.768295],
        .

In [ ]:
array([[ 0.8695588 , -0.10210263, -0.70709586, ...,  0.35408315,
        -0.7213362 , -0.683963  ],
       [-0.89677817,  0.7956062 , -1.2477107 , ...,  0.52501583,
         0.5512355 ,  1.4323367 ],
       [ 1.4528289 , -1.9335874 ,  0.3995513 , ...,  0.5208921 ,
        -0.6019875 ,  0.8595276 ],
       ...,
       [ 0.24775645, -0.19322538, -1.0393876 , ..., -1.5815692 ,
         1.4920877 , -0.3311486 ],
       [ 0.2905283 ,  0.01512055, -0.62715375, ...,  0.47839084,
         0.47361645,  0.84168893],
       [ 0.3841245 , -1.3143691 , -1.1167487 , ...,  0.52404374,
         0.9821101 , -0.7097833 ]], dtype=float32)